In [1]:
import random

import torch
import pytorch_lightning as pl
from utils.optoforce_data_loader import load_data
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import *
from utils.preprocessing import remove_padding
from models.encoder_decoder_lstm import *

In [2]:
PATH_TO_DIR = 'C:/Users/sonia/OneDrive - Queen Mary, University of London/Action-Segmentation-Project'
# X_data, y_data, _ = preprocess_dataset(PATH_TO_DIR)
# train_loader, val_loader, test_loader = load_data(X_data,y_data)

In [3]:
#X, y = next(iter(train_loader))

In [4]:
#y[0]

tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  2,  2,  2,  5,
         5,  5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [3]:
class EncoderLSTM(nn.Module):
    """ Encodes tactile time series data """

    def __init__(self, n_features =3, hidden_size=100, n_layers=1, n_classes=6):
        super().__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_classes = n_classes

        self.lstm = nn.LSTM(input_size=self.n_features, hidden_size=self.hidden_size,
                            num_layers=self.n_layers, batch_first=True)

    def forward(self, x):
        batch_size = x.shape[0]

        output, (h_n, c_n) = self.lstm(x)
        # print(f"encoder hidden shape {h_n.shape}")
        # print(f"encoder cell shape {c_n.shape}")
        #
        # print(f"output shape {output.shape}")
        return h_n,c_n

    # def _init_states(self, batch_size):
    #     if torch.cuda.is_available():
    #         h0 = torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True, device="cuda")
    #         # c0 = torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True, device="cuda")
    #
    #     else:
    #         h0 = torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True)
    #         # c0 = torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True)
    #
    #     return h0, c0

In [4]:
class DecoderLSTM(torch.nn.Module):
    def __init__(self, hidden_size =100, n_classes=6):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_classes = n_classes

        self.lstm = nn.LSTM(1, self.hidden_size)
        self.linear = nn.Linear(in_features=self.hidden_size, out_features=self.n_classes)


    def forward(self, x, hidden, cell):
        """"
        hidden - the final hidden state from the encoder model is the context vector of the source sequence.
        x - is the target ouput

        """
        #x = x.unsqueeze(0)
        #print("adding extra dimension in decoder input")
        #x = torch.LongTensor(x.view(1,1,1))
        #
        # print(f"decoder input shape: {x.shape}")
        #lstm input size: (seq_len, batch,n_features) = (1,1,1)
        # print(f"decoder input: {x}")
        # print(f"decoder hidden input shape: {hidden.shape}")
        # print(f"decoder cell input shape : {cell.shape}")
        output, (hidden,cell) = self.lstm(x, (hidden, cell))
        #output shape: (1,1,100)

        # print(f"decoder output shape: {output.shape}")
        # print(f"decoder hidden shape: {hidden.shape}")
        # print(f"decoder cell shape: {hidden.shape}")


        flatten_output = output.view(-1, output.shape[2])

        # print(f"flatten output shape:  {flatten_output.shape} ")
        #flatten_output shape: (1, 100)
        logits = self.linear(flatten_output)
        #shape of logits: (1,6)

        # print(f"logits shape:{logits.shape}")

        return logits, hidden,cell



In [7]:
# t = torch.rand(1,3)
# print(t)
# out = t.argmax(-1)
# print(out)

In [5]:
class EncoderDecoderLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = EncoderLSTM()
        self.decoder = DecoderLSTM()

        assert self.encoder.hidden_size == self.decoder.hidden_size


    def forward(self,x,y, teacher_forcing_ratio, n_classes = 6):

        batch_size = y.shape[0]
        optoforce_seq_len = y.shape[1]

        outputs = torch.zeros(batch_size,optoforce_seq_len,n_classes)

        hidden, cell = self.encoder(x)
        # print("context vector shape")
        # print(hidden.shape)
        #decoder_input = torch.zeros((1,1,1)) #not sure
        # print("beginning decoder input")
        # print(decoder_input.shape)
        decoder_input = y[0][0]
        decoder_input = decoder_input.type(torch.float32)
        decoder_input = decoder_input.view(1,1,1)
        for t in range(1,optoforce_seq_len):
            #print(t)
            # print(f"decoder input shape at time {t} = {decoder_input.shape}")
            # print(f"h_x.shape {hidden.shape}")
            # print(f"cell.shape {cell.shape}")
            #print(f"current t {t}")
            output, hidden, cell  = self.decoder(decoder_input, hidden, cell)
            #output shape (1,6)

            # print(f"output predicted at time {t} = {output}")
            outputs[0][t-1] = output

            teacher_force = random.random() < teacher_forcing_ratio
            # print(f"teacher forcing present at time {t} = {teacher_force}")
            top_pred = output.argmax(-1)
            #print(f"top predicition at time {t} = {top_pred}")
            decoder_input = y[0][t] if teacher_force else top_pred

            #print(f"getting decoder input from time {t}")
            decoder_input = decoder_input.type(torch.float32)
            decoder_input = decoder_input.view(1,1,1)
            #print(f"next input at time {t} = {decoder_input.shape}, {decoder_input}")
            #


        return outputs

In [9]:
# encoder = EncoderLSTM()
# decoder = DecoderLSTM()

In [10]:
# hidden,cell = encoder(X)

In [11]:
# t_1 = y[0][1]
# t_1 = t_1.type(torch.float32)
# t_1 = t_1.view(1,1,1)
# print(t_1)
#
# t_1.dtype

In [12]:
# t_0 = torch.zeros([1,1,1]) #seq_len x batch_size x n_features
# t_0.dtype


In [13]:
# d_logits,d_hidden,d_cell = decoder(t_0,hidden,cell)

In [14]:
# d_logits,d_hidden,d_cell = decoder(t_1,d_hidden,d_cell)

In [15]:
# y.shape[0]

In [16]:
# outputs = torch.zeros(y.shape[0],y.shape[1],6)
#
# outputs[0][1] = d_logits

In [17]:
# print(outputs)

In [6]:
from utils.optoforce_datamodule import OpToForceDataModule, KFoldLoop
class LitEncoderDecoderLSTM(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.encoder_decoder_model = EncoderDecoderLSTM()
        self.loss_module = nn.CrossEntropyLoss(ignore_index=-1)
        self.train_acc = Accuracy(ignore_index=-1)
        self.val_acc = Accuracy(ignore_index=-1)
        self.test_acc = Accuracy(ignore_index=-1)

    def forward(self,X,y,teacher_forcing):
        logits = self.encoder_decoder_model(X,y,teacher_forcing)
        return logits

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer


    def training_step(self, batch, batch_idx):
        X, y = batch

        logits, loss = self._get_preds_and_loss(X,y,teacher_forcing=0.5)
        train_perplexity = torch.exp(loss)
        #logits shape : (n_timesteps, n_classes)
        self.train_acc(logits,y.squeeze(0)) #remove batch dimension
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train_PPL', train_perplexity, on_step=False, on_epoch=True, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        X, y = batch

        logits, val_loss = self._get_preds_and_loss(X,y,teacher_forcing=0.0)

        val_perplexity = torch.exp(val_loss)
        self.val_acc(logits, y.squeeze(0))

        self.log('val_loss', val_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)

        self.log('val_PPL', val_perplexity, on_step=False, on_epoch=True, prog_bar=True)
        #return val_loss


    def test_step(self, batch, batch_idx):
        X, y = batch

        logits, test_loss = self._get_preds_and_loss(X,y,teacher_forcing=0.0)

        test_perplexity = torch.exp(test_loss)
        self.test_acc(logits,y.squeeze(0)) #remove the batch dimension

        self.log("test_loss", test_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_acc", self.test_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('test_PPL', test_perplexity, on_step=False, on_epoch=True, prog_bar=True)
        #return test_loss

    def _get_preds_and_loss(self,X,y, teacher_forcing):

        logits = self(X,y,teacher_forcing)
        logits = logits.squeeze(0) #remove the batch dimension
        loss = self.loss_module(logits,y.squeeze(0))

        return logits , loss

In [28]:
# encoder_decoder = EncoderDecoderLSTM(encoder,decoder)

In [29]:
# outputs = encoder_decoder(X,y,0.5)

In [30]:
#outputs.shape

In [31]:
#outputs[0]

In [32]:
#outputs[0][0:101]

In [33]:
# element = 99
#
# y = [100,200,300]
#
# for i in range(1,len(y)+1):
#     print(i)
#     input = element*2
#     print(input)
#     element = y[i-1]

In [9]:
#trainer = pl.Trainer(default_root_dir=f"{PATH_TO_DIR}/checkpoints",  gpus=0, max_epochs=20, deterministic=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
#model = LitEncoderDecoderLSTM()

In [7]:
#trainer.fit(model, train_loader, val_loader)

In [8]:
#trainer.test(dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                  | Type               | Params
-------------------------------------------------------------
0 | encoder_decoder_model | EncoderDecoderLSTM | 83.8 K
1 | loss_module           | CrossEntropyLoss   | 0     
2 | train_acc             | Accuracy           | 0     
3 | val_acc               | Accuracy           | 0     
4 | test_acc              | Accuracy           | 0     
-------------------------------------------------------------
83.8 K    Trainable params
0         Non-trainable params
83.8 K    Total params
0.335     Total estimated model params size (MB)


STARTING FOLD 0


C:\Users\sonia\anaconda3\envs\action-seg\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\sonia\anaconda3\envs\action-seg\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
C:\Users\sonia\anaconda3\envs\action-seg\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_PPL             6.544190406799316
        test_acc            0.27659574151039124
        test_loss           1.8642303943634033
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    average test_acc        0.27234041690826416
    average_test_loss       1.7213537693023682
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
PATH_TO_DIR = 'C:/Users/sonia/OneDrive - Queen Mary, University of London/Action-Segmentation-Project'
pl.seed_everything(42)
X_data, y_data, labels_map = preprocess_dataset(PATH_TO_DIR)
model = LitEncoderDecoderLSTM()
datamodule = OpToForceDataModule(X_data,y_data)

trainer = pl.Trainer(
    max_epochs=10,
    limit_train_batches=2,
    limit_val_batches=2,
    limit_test_batches=2,
    num_sanity_val_steps=0,
    #devices=2,
    gpus=0,
    accelerator="auto",
    #strategy="ddp",

)

internal_fit_loop = trainer.fit_loop
trainer.fit_loop = KFoldLoop(num_folds=4,export_path='testing_kfold')
trainer.fit_loop.connect(internal_fit_loop)
trainer.fit(model, datamodule)
#trainer.test(datamodule)

Global seed set to 42
